In [ ]:
import os
import sys

In [ ]:
def instal_packages():
    # setup packages
    !pip install tensorflow-model-optimization
    !pip install PyYAML==5.4.1

In [ ]:
def setup_project(project_path, model):
    def download_barcode_detection():
        print('Downloading barcode detection module...')
        # Download source code.
        if not os.path.exists(os.path.join(os.getcwd(), '/BarcodeScanner')):
            !git clone --depth 1 https://github.com/SavanK/BarcodeScanner
        os.chdir('BarcodeScanner/')
        !git pull
        os.chdir('..')
    
    os.chdir(project_path)
    download_barcode_detection()
    os.chdir('BarcodeScanner/barcode_detection/')
    print(f'CWD: {os.getcwd()}')

    %run ./setup_efficientdet.ipynb
    install_efficientdet()
    ckpt_path = download_efficientdet_model(model)
    print(f'Model downloaded at {ckpt_path}')

In [ ]:
def setup_data():
    print(f'Setting up data...')
    %run ./generate_tfrecord.ipynb
    TFRECORDS_DIR = os.path.join(os.getcwd(), 'tfrecords/')
    DATASET_DIR = os.path.join(os.getcwd(), 'Muenster_Barcode_Database/N95-2592x1944_scaledTo640x480bilinear')
    ANNOTATIONS_PATH = os.path.join(os.getcwd(), 'Muenster_Barcode_Database/annotations.csv')

    create_train_val_tfrecords(DATASET_DIR, ANNOTATIONS_PATH, TFRECORDS_DIR, 25)
    print(f'Created tfrecords for train and validation data at {TFRECORDS_DIR}')
    
    return TFRECORDS_DIR

In [ ]:
MODEL = 'efficientdet-d0'

install_required_packages()
setup_project(project_path='/content/')
tfrecords_path = setup_data()
tfrecords_dir = os.path.basename(tfrecords_path)

%run ./retrain_efficientdet_for_barcode.ipynb
retrain_efficientdet_for_barcode(MODEL, tfrecords_dir)